# --- Day 22: Wizard Simulator 20XX ---

Little Henry Case decides that defeating bosses with swords and stuff is boring. Now he's playing the game with a wizard. Of course, he gets stuck on another boss and needs your help again.

In this version, combat still proceeds with the player and the boss taking alternating turns. The player still goes first. Now, however, you don't get any equipment; instead, you must choose one of your spells to cast. The first character at or below 0 hit points loses.

Since you're a wizard, you don't get to wear armor, and you can't attack normally. However, since you do magic damage, your opponent's armor is ignored, and so the boss effectively has zero armor as well. As before, if armor (from a spell, in this case) would reduce damage below 1, it becomes 1 instead - that is, the boss' attacks always deal at least 1 damage.

On each of your turns, you must select one of your spells to cast. If you cannot afford to cast any spell, you lose. Spells cost mana; you start with 500 mana, but have no maximum limit. You must have enough mana to cast a spell, and its cost is immediately deducted when you cast it. Your spells are Magic Missile, Drain, Shield, Poison, and Recharge.

- Magic Missile costs 53 mana. It instantly does 4 damage.
- Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
- Shield costs 113 mana. It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
- Poison costs 173 mana. It starts an effect that lasts for 6 turns. At the start of each turn while it is active, it deals the boss 3 damage.
- Recharge costs 229 mana. It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.

Effects all work the same way. Effects apply at the start of both the player's turns and the boss' turns. Effects are created with a timer (the number of turns they last); at the start of each turn, after they apply any effect they have, their timer is decreased by one. If this decreases the timer to zero, the effect ends. You cannot cast a spell that would start an effect which is already active. However, effects can be started on the same turn they end.

For example, suppose the player has 10 hit points and 250 mana, and that the boss has 13 hit points and 8 damage:

-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --
- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage.

-- Player turn --
- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
Now, suppose the same initial conditions, except that the boss has 14 hit points instead:

-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 14 hit points
Player casts Recharge.

-- Boss turn --
- Player has 10 hit points, 0 armor, 21 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 4.
Boss attacks for 8 damage!

-- Player turn --
- Player has 2 hit points, 0 armor, 122 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 3.
Player casts Shield, increasing armor by 7.

-- Boss turn --
- Player has 2 hit points, 7 armor, 110 mana
- Boss has 14 hit points
Shield's timer is now 5.
Recharge provides 101 mana; its timer is now 2.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 211 mana
- Boss has 14 hit points
Shield's timer is now 4.
Recharge provides 101 mana; its timer is now 1.
Player casts Drain, dealing 2 damage, and healing 2 hit points.

-- Boss turn --
- Player has 3 hit points, 7 armor, 239 mana
- Boss has 12 hit points
Shield's timer is now 3.
Recharge provides 101 mana; its timer is now 0.
Recharge wears off.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 2 hit points, 7 armor, 340 mana
- Boss has 12 hit points
Shield's timer is now 2.
Player casts Poison.

-- Boss turn --
- Player has 2 hit points, 7 armor, 167 mana
- Boss has 12 hit points
Shield's timer is now 1.
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 167 mana
- Boss has 9 hit points
Shield's timer is now 0.
Shield wears off, decreasing armor by 7.
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 1 hit point, 0 armor, 114 mana
- Boss has 2 hit points
Poison deals 3 damage. This kills the boss, and the player wins.

You start with 50 hit points and 500 mana points. The boss's actual stats are in your puzzle input. What is the least amount of mana you can spend and still win the fight? (Do not include mana recharge effects as "spending" negative mana.)

In [36]:
# Define the initial stats (including player stats and timers)
bossStats = {'HP': 55, 'Damage': 8}
jonoStats = {'HP': 50, 'Mana': 500}

In [37]:
# Define a function which identifies the potential next move in a new round, based on any given sequence of previous spells
def newRound(oldSequences, topScore):
    # Once assessed, unresolved (no winner yet) fight sequences will be recorded for the next round
    # I used a dictionary with the value as the total mana to help with troubleshooting
    newSequences = {}
    # For each unresolved fight sequence from the previous round, try each combination of possible spells
    for sequence in oldSequences:  
        newSequence = ''
        # Using another function (manaLeft), check if you have enough mana to cast the spell - in this case Magic Missile ('m')
        if manaLeft(sequence, bossStats, jonoStats) >= 53:
            # Using another function (newSpell), see if the new sequence results in a winner and if not, carry to next round
            newSequences, topScore = newSpell(sequence, bossStats, jonoStats, 'm', topScore, newSequences)                
        if manaLeft(sequence, bossStats, jonoStats) >= 73:
            newSequences, topScore = newSpell(sequence, bossStats, jonoStats, 'd', topScore, newSequences)
        # For spells with a counter, check if there is another instance active as well as the mana check to check legality       
        if manaLeft(sequence, bossStats, jonoStats) >= 113 and 's' not in sequence[-2:]:
            newSequences, topScore = newSpell(sequence, bossStats, jonoStats, 's', topScore, newSequences)                
        if manaLeft(sequence, bossStats, jonoStats) >= 173 and 'p' not in sequence[-2:]:
            newSequences, topScore = newSpell(sequence, bossStats, jonoStats, 'p', topScore, newSequences)                
        if manaLeft(sequence, bossStats, jonoStats) >= 229 and 'r' not in sequence[-2:]:
            newSequences, topScore = newSpell(sequence, bossStats, jonoStats, 'r', topScore, newSequences)
    # Return the unresolved sequences for the next round and the current top score (i.e. lowest mana for a win)
    return newSequences, topScore

# For the manaLeft function, subtract the cost of all spells used from your start mana, then add recovered mana from Recharge
def manaLeft(sequence, bossStats, jonoStats):
    mana = jonoStats['Mana']   
    # Use a for loop to check each spell case so far
    for attack in sequence:
        if attack == 'm':
            mana -= 53
        elif attack == 'd':
            mana -= 73
        elif attack == 's':
            mana -= 113
        elif attack == 'p':
            mana -= 173
        elif attack == 'r':
            mana -= 229            
    # If the last spell or second last spell was a Recharge, it won't have fully resolved yet, which needs accounting for
    if sequence[-1] == 'r':
        mana += ((sequence[:-1].count('r')) * 505 + 101)
    elif len(sequence) >= 2 and sequence[-2] == 'r':
        mana += ((sequence[:-2].count('r')) * 505 + 303)             
    # Otherwise you add the full 5 x 101 effect from the spell
    else:
        mana += ((sequence.count('r')) * 505)              
    return mana

# The newSpell function adds the new spell to the existing sequence and then calculates if the player or boss will win this round
# If no-one wins, it passes the sequence to the next round
def newSpell(sequence, bossStats, jonoStats, spell, topScore, newSequences):
    newSequence = sequence + spell
    # print(sequence, spell, newSequence)
    # Use another function to check for the win. If the player wins and the mana cost is the best yet, set the new top score!
    if winCheck(newSequence, bossStats, jonoStats) == True and manaUsed(newSequence, bossStats, jonoStats) <= topScore:
        # Use another function to determine the magic used
        topScore = manaUsed(newSequence, bossStats, jonoStats)
    # Use another function to check for the loss.
    elif deadCheck(newSequence, bossStats, jonoStats) == True:
        pass
    # Otherwise pass the sequence to the next round
    elif manaUsed(newSequence, bossStats, jonoStats) <= topScore:
        newSequences[newSequence] = manaUsed(newSequence, bossStats, jonoStats)
    return newSequences, topScore

# To check the win, check for the total damage done from the string, accounting for duration of poison
def winCheck(sequence, bossStats, jonoStats):
    win = False
    # If poison is within last 3 spells it won't have fully resolved and must be accounted for
    if sequence[-1] == 'p':
        if sequence.count('m') * 4 + sequence.count('d') * 2 + sequence.count('p') * 18 - 15 >= bossStats['HP']:
            win = True
    elif len(sequence) >= 2 and sequence[-2] == 'p':
        if sequence.count('m') * 4 + sequence.count('d') * 2 + sequence.count('p') * 18 - 9 >= bossStats['HP']:
            win = True
    elif len(sequence) >= 3 and sequence[-3] == 'p':
        if sequence.count('m') * 4 + sequence.count('d') * 2 + sequence.count('p') * 18 - 3 >= bossStats['HP']:
            win = True
    # Otherwise it does 4 for each missile, 2 for each drain and 6 x 3 for each completed poison
    else: 
        if sequence.count('m') * 4 + sequence.count('d') * 2 + sequence.count('p') * 18 >= bossStats['HP']:
            win = True
    return win

# To check the loss, check for the total damage recieved from the boss, accounting for duration shields
def deadCheck(sequence, bossStats, jonoStats):
    dead = False
    # If shield was within the last 2 spells it wont have fully resolved and must be accounted for
    if sequence[-1] == 's':
        if ((sequence[:-1].count('s')) * 3 + 1) * (bossStats['Damage'] - 7) + (len(sequence) - ((sequence[:-1].count('s')) * 3 + 1)) * bossStats['Damage'] - sequence.count('d') * 2 >= jonoStats['HP']:
            dead = True
    elif len(sequence) >= 2 and sequence[-2] == 's':
        if ((sequence[:-2].count('s')) * 3 + 2) * (bossStats['Damage'] - 7) + (len(sequence) - ((sequence[:-2].count('s')) * 3 + 2)) * bossStats['Damage'] - sequence.count('d') * 2 >= jonoStats['HP']:
            dead = True              
    # Otherwise take Damage - 7 for each active sheild turn (3x) and damage for the rest of the turns
    else:
        if ((sequence.count('s')) * 3) * (bossStats['Damage'] - 7) + (len(sequence) - (sequence.count('s')) * 3) * bossStats['Damage'] - sequence.count('d') * 2 >= jonoStats['HP']:
            dead = True
    return dead

# To check magic used, just add all the spell costs in the sequence
def manaUsed(sequence, bossStats, jonoStats):
    mana = 0   
    for attack in sequence:
        if attack == 'm':
            mana += 53
        elif attack == 'd':
            mana += 73
        elif attack == 's':
            mana += 113
        elif attack == 'p':
            mana += 173
        elif attack == 'r':
            mana += 229    
    return mana

In [38]:
# Set the initial possible initial attack sequences, and an arbitrarily high top score
attackSequences = {'m':53, 'd':73, 's':113, 'p':173, 'r':229}
topScore = 10000
counter = 0

# Until no lower mana sequences are left to test, feed new sequenced into the newRound function
while len(attackSequences) >= 1:
    counter += 1
    attackSequences, topScore = newRound(attackSequences, topScore)
    print('After round', counter, 'there were', len(attackSequences), 'possible sequences.')
    print('TopScore is ',topScore)

After round 1 there were 22 possible sequences.
TopScore is  10000
After round 2 there were 84 possible sequences.
TopScore is  10000
After round 3 there were 310 possible sequences.
TopScore is  10000
After round 4 there were 1028 possible sequences.
TopScore is  10000
After round 5 there were 3241 possible sequences.
TopScore is  10000
After round 6 there were 8778 possible sequences.
TopScore is  10000
After round 7 there were 29552 possible sequences.
TopScore is  10000
After round 8 there were 35799 possible sequences.
TopScore is  953
After round 9 there were 16655 possible sequences.
TopScore is  953
After round 10 there were 8229 possible sequences.
TopScore is  953
After round 11 there were 0 possible sequences.
TopScore is  953


# --- Part Two ---

On the next run through the game, you increase the difficulty to hard.

At the start of each player turn (before any other effects apply), you lose 1 hit point. If this brings you to or below 0 hit points, you lose.

With the same starting stats for you and the boss, what is the least amount of mana you can spend and still win the fight?

In [39]:
# In effect, this condition just lowers cour starting HP by 1 (from before 1st turn) 
# and increases the boss attack by 1 (happens immediately after boss attack)
bossStats = {'HP': 55, 'Damage': 9}
jonoStats = {'HP': 49, 'Mana': 500}

In [40]:
attackSequences = {'m':53, 'd':73, 's':113, 'p':173, 'r':229}
topScore = 10000
counter = 0

while len(attackSequences) >= 1:
    counter += 1
    attackSequences, topScore = newRound(attackSequences, topScore)
    print('After round', counter, 'there were', len(attackSequences), 'possible sequences.')
    print('TopScore is ',topScore)

After round 1 there were 22 possible sequences.
TopScore is  10000
After round 2 there were 84 possible sequences.
TopScore is  10000
After round 3 there were 310 possible sequences.
TopScore is  10000
After round 4 there were 1028 possible sequences.
TopScore is  10000
After round 5 there were 2492 possible sequences.
TopScore is  10000
After round 6 there were 7440 possible sequences.
TopScore is  10000
After round 7 there were 22159 possible sequences.
TopScore is  10000
After round 8 there were 38129 possible sequences.
TopScore is  1289
After round 9 there were 71178 possible sequences.
TopScore is  1289
After round 10 there were 60495 possible sequences.
TopScore is  1289
After round 11 there were 42573 possible sequences.
TopScore is  1289
After round 12 there were 18232 possible sequences.
TopScore is  1289
After round 13 there were 0 possible sequences.
TopScore is  1289
